In [1]:
# Import
import pandas as pd
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read in data
basepath = 'C:/Users/Kelsey/Desktop/jobs/TheTrevorProject/Project/data/'
bdata = pd.read_csv(basepath + 'barackobama.csv')
cdata = pd.read_csv(basepath + 'calvinstowell.csv')
kdata = pd.read_csv(basepath + 'kimkardashian.csv')

# Combine data
alldata = pd.concat([bdata, cdata, kdata])
print(alldata.shape)
alldata.head()

(6785, 5)


,created_at,handle,retweet_count,text,id
0,Mon Jun 03 20:59:36 +0000 2019,barackobama,2873,What a life. American history has always been ...,1135652276515823617
1,Thu May 30 20:12:33 +0000 2019,barackobama,13781,"A story worth sharing: Twelve years ago, Brian...",1134190885036535809
2,Thu May 30 15:17:03 +0000 2019,barackobama,4745,Exciting to see the faces of Colombia's future...,1134116519066165249
3,Mon May 27 15:26:55 +0000 2019,barackobama,43244,"On Memorial Day, we remember all those who gav...",1133031835880099840
4,Thu May 23 14:07:16 +0000 2019,barackobama,15144,Great to get out there and take a few cuts at ...,1131562241478864896


In [3]:
# Split dataframes
# Feel like i should sample larger classes & test, but maybe not to Obama size since he is gonna be the easiest to detect?
# 1 - undersample calvin & kim's and do split on 1000 tweet corpuses
# 2 - cross validation on samples & test
# 3 - Run 2 class classification models w/ calvin & kim only and see if helps differentiate
# 4? - Markov chain obama and create more tweets, hah

## Option 1 - 80/20 of 1000 tweet samples
# Limit Calvin & Kim to random 1000
cdata_lim = cdata.sample(n=1000, random_state=1)
cdata_left = cdata.drop(cdata_lim.index)
kdata_lim = kdata.sample(n=1000, random_state=1)
kdata_left = kdata.drop(kdata_lim.index)

# Split & Combine
tb = bdata.sample(frac=0.8, random_state=1)
vb = bdata.drop(tb.index)
tc = cdata_lim.sample(frac=0.8, random_state=1)
vc = cdata_left.sample(n=200, random_state=1)
tk = kdata_lim.sample(frac=0.8, random_state=1)
vk = kdata_left.sample(n=200, random_state=1)

train1 = pd.concat([tb, tc, tk])
valid1 = pd.concat([vb, vc, vk])
all1 = pd.concat([bdata, cdata_lim, kdata_lim])


## Option 2 - different sample
# Limit Calvin & Kim to random 1000
cdata_lim = cdata.sample(n=1000, random_state=2)
cdata_left = cdata.drop(cdata_lim.index)
kdata_lim = kdata.sample(n=1000, random_state=2)
kdata_left = kdata.drop(kdata_lim.index)

# Split & Combine
tb = bdata.sample(frac=0.8, random_state=2)
vb = bdata.drop(tb.index)
tc = cdata_lim.sample(frac=0.8, random_state=2)
vc = cdata_left.sample(n=200, random_state=2)
tk = kdata_lim.sample(frac=0.8, random_state=2)
vk = kdata_left.sample(n=200, random_state=2)

train2 = pd.concat([tb, tc, tk])
valid2 = pd.concat([vb, vc, vk])
all2 = pd.concat([bdata, cdata_lim, kdata_lim])




# Current set
train = train2
valid = valid2
alldat =  all2

# Shuffle
train = train.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)
alldat = alldat.sample(frac=1).reset_index(drop=True)

print(train.shape)
print(valid.shape)
print(alldat.shape)

(2400, 5)
(600, 5)
(3000, 5)


In [9]:
import re
import nltk

stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['wouldnt', 'wont', 'werent', 'wasnt', 'shouldnt', 'neednt', 'isnt', 'havent', 'hasnt', 'hadnt', 'ive','doesnt', 'didnt', 'couldnt', 'arent', 'aint', 'amp'])

In [10]:
import string
string.punctuation
punctuation = string.punctuation + '’' + '–' + '“' + '”'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~’–“”'

In [11]:
# Text clean/transform function

def textonly(text):
    # Remove URLS
    text = re.sub('https?:\/\/.*', '', text)
    # Remove Punctuation
    text = re.sub('—', ' ', text)
    text  = "".join([char for char in text if char not in punctuation])
    # Lowercase
    text = text.lower()
    # Remove Stopwords, Tokenize
    return [word for word in nltk.word_tokenize(text) if word not in stopwords]

In [12]:
# Important bag of words features from naive model
obamaimpfeat = ['americans', 'make', 'today', 'people', 'fair', 'vote', 'us', 'hearing', 'doyourjob', 'climate', 'supreme', 'change', 'court', 'garland', 'judge', 'rt', 'leaders', 'senate', 'obama', 'president']
calvinimpfeat = ['still', 'happy', 'know', 'literally', 'love', 'stevemorris', 'yall', 'good', 'legend', 'omg', 'trump', 'really', 'one', 'dont', 'gay', 'people', 'like', 'yashar', 'im', 'rt']
kimimpfeat = ['day', 'classic', '✨', 'kkwxmario', 'available', 'love', 'lip', 'kuwtk', 'today', '12pm', 'collection', 'get', 'happy', 'kimkardashian', 'kkwfragrance', 'pst', 'new', 'shop', 'kkwbeauty', 'rt']

wordfeats = list(set(obamaimpfeat + calvinimpfeat + kimimpfeat))
len(wordfeats)  

54

In [31]:
# pd.options.display.max_colwidth = 500
# # Dataframe with features
# import numpy as np

# X = alldat[['text', 'retweet_count']]

# X['text_bascln'] = X['text'].apply(lambda x: textonly(x))

# for w in wordfeats:
#     #X[w] = 0
#     print(w)
#     print(X['text_bascln'])
#     print(w in X['text_bascln'])
#     X[w] = np.where(w in X['text_bascln'], 1, 0)
    
# X.head(10)

c:\users\kelsey\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [33]:
from sklearn.preprocessing import LabelEncoder

# Encode label
y = alldat['handle']
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
# alpha order - 0=barack, 1 = calvin, 2=kim

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(wordfeats)
print(vectorizer.get_feature_names())
# ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
# >>> print(X.toarray())  
# [[0 1 1 1 0 0 1 0 1]
#  [0 2 0 1 0 1 1 0 1]
#  [1 0 0 1 1 0 1 1 1]
#  [0 1 1 1 0 0 1 0 1]]

['12pm', 'americans', 'available', 'change', 'classic', 'climate', 'collection', 'court', 'day', 'dont', 'doyourjob', 'fair', 'garland', 'gay', 'get', 'good', 'happy', 'hearing', 'im', 'judge', 'kimkardashian', 'kkwbeauty', 'kkwfragrance', 'kkwxmario', 'know', 'kuwtk', 'leaders', 'legend', 'like', 'lip', 'literally', 'love', 'make', 'new', 'obama', 'omg', 'one', 'people', 'president', 'pst', 'really', 'rt', 'senate', 'shop', 'stevemorris', 'still', 'supreme', 'today', 'trump', 'us', 'vote', 'yall', 'yashar']


In [35]:
X = alldat['text']

from sklearn.model_selection import train_test_split

# 80-20 splitting the dataset (80%->Training and 20%->Validation)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=1)

In [36]:
# defining the bag-of-words transformer on the text-processed corpus # i.e., text_process() declared in II is executed...
#bow_transformer=CountVectorizer(analyzer=textonly).fit(X_train)
bow_transformer=CountVectorizer(analyzer=textonly).fit(wordfeats)
# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_train=bow_transformer.transform(X_train)#ONLY TRAINING DATA
# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_test=bow_transformer.transform(X_test)#TEST DATA

In [37]:
from sklearn.naive_bayes import MultinomialNB

# instantiating the model with Multinomial Naive Bayes..
model = MultinomialNB()
# training the model...
model = model.fit(text_bow_train, y_train)

In [38]:
# Training accuracy
model.score(text_bow_train, y_train)

0.58875

In [39]:
testcase = "Don’t make time for people who don’t make time for you. https://t.co/5iSinrlCz1"
bow_transformer.transform([testcase])
#print(loaded_model.predict_proba(bow_transformer.transform([testcase]))) 

<1x54 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [41]:
bow_transformer.get_feature_names()

['12pm',
 'americans',
 'available',
 'change',
 'classic',
 'climate',
 'collection',
 'court',
 'day',
 'dont',
 'doyourjob',
 'fair',
 'garland',
 'gay',
 'get',
 'good',
 'happy',
 'hearing',
 'im',
 'judge',
 'kimkardashian',
 'kkwbeauty',
 'kkwfragrance',
 'kkwxmario',
 'know',
 'kuwtk',
 'leaders',
 'legend',
 'like',
 'lip',
 'literally',
 'love',
 'make',
 'new',
 'obama',
 'omg',
 'one',
 'people',
 'president',
 'pst',
 'really',
 'rt',
 'senate',
 'shop',
 'stevemorris',
 'still',
 'supreme',
 'today',
 'trump',
 'us',
 'vote',
 'yall',
 'yashar',
 '✨']

In [42]:
import pickle
 
with open(basepath + 'words.pkl', 'wb') as f:
    pickle.dump(bow_transformer.get_feature_names(), f)

In [44]:
with open(basepath + 'words.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

mynewlist

['12pm',
 'americans',
 'available',
 'change',
 'classic',
 'climate',
 'collection',
 'court',
 'day',
 'dont',
 'doyourjob',
 'fair',
 'garland',
 'gay',
 'get',
 'good',
 'happy',
 'hearing',
 'im',
 'judge',
 'kimkardashian',
 'kkwbeauty',
 'kkwfragrance',
 'kkwxmario',
 'know',
 'kuwtk',
 'leaders',
 'legend',
 'like',
 'lip',
 'literally',
 'love',
 'make',
 'new',
 'obama',
 'omg',
 'one',
 'people',
 'president',
 'pst',
 'really',
 'rt',
 'senate',
 'shop',
 'stevemorris',
 'still',
 'supreme',
 'today',
 'trump',
 'us',
 'vote',
 'yall',
 'yashar',
 '✨']

In [8]:
# Create features

pd.options.display.max_colwidth = 500

train['text_bascln'] = train['text'].apply(lambda x: textonly(x))
train[['text', 'text_bascln']].head(10)


df.drop('Col3', 1).join(df.Col3.str.join('|').str.get_dummies())
X_train.join(df.Col3.str.join('|').str.get_dummies())

  (0, 10)	1
  (1, 43)	1
  (2, 37)	1
  (3, 50)	1
  (4, 22)	1
  (5, 45)	1
  (6, 15)	1
  (7, 30)	1
  (9, 40)	1
  (10, 13)	1
  (11, 46)	1
  (12, 17)	1
  (13, 39)	1
  (14, 36)	1
  (15, 44)	1
  (16, 18)	1
  (17, 42)	1
  (18, 1)	1
  (19, 11)	1
  (20, 24)	1
  (21, 14)	1
  (22, 0)	1
  (23, 19)	1
  (24, 23)	1
  (25, 25)	1
  :	:
  (29, 49)	1
  (30, 41)	1
  (31, 48)	1
  (32, 6)	1
  (33, 3)	1
  (34, 33)	1
  (35, 26)	1
  (36, 20)	1
  (37, 5)	1
  (38, 29)	1
  (39, 4)	1
  (40, 8)	1
  (41, 51)	1
  (42, 32)	1
  (43, 12)	1
  (44, 7)	1
  (45, 47)	1
  (46, 16)	1
  (47, 9)	1
  (48, 27)	1
  (49, 38)	1
  (50, 34)	1
  (51, 52)	1
  (52, 21)	1
  (53, 31)	1
